In [1]:
import os, sys, pickle
import matplotlib.pyplot as plt
import numpy as np
import collections
%matplotlib inline 
sys.path.append('/home/jp/projects/snoplus/python_modules')
import jp_mpl as jplot
from scipy import ndimage
from scipy import signal
from matplotlib.colors import LogNorm
from detect_peaks import detect_peaks
import pickle, scipy
from copy import deepcopy

## Fitting class

In [2]:
class FitLBpos(object):
    def __init__(self,
                 data = None,
                 error = None,
                 pmt_xyz = None,
                 pmtbool = None,
                 psup_radius = 8390.,
                 water_n = 1.34389,
                 print_call = True):
        self.c            = 0.299792458*1000 # mm/ns
        self.data    = deepcopy(data)
        self.pmt_xyz = deepcopy(pmt_xyz)
        self.pmtbool = deepcopy(pmtbool)
        self.pmt_r   = np.linalg.norm(pmt_xyz,axis=1)
        self.header_done = False

        self.R = psup_radius
        #self.water_c = c/water_n
        self.print_call = print_call
        
        self.pmtbool[self.data == 0] = False
        
        if error == None:
            self.error = np.ones_like(data)
        else:
            self.error = error + 1 # Adding one ns for everything
    
    def print_eval_header(self):
        print 'FCN \t\t u \t v \t w \t n'
        self.header_done = True
        
    def print_eval(self, value, u, v, w, n):
        if not self.header_done:
            self.print_eval_header()
        print value, '\t', u, '\t', v, '\t', w, '\t', n
        
    def __call__(self, u, v, w, n):
        this_pos = np.array([ u, v, w])
        water_c = self.c/n
        tdiff = 2*self.R/water_c* \
                (1+np.dot(self.pmt_xyz,this_pos)/(self.pmt_r*self.R))
        
        delta = (tdiff - self.data)[self.pmtbool]**2/self.error[self.pmtbool]**2
        delta = np.sum(delta)
        
        if self.print_call:
            self.print_eval(delta, u, v, w, n)
        
        return delta

## PMT Information

In [3]:
pmt_info = pickle.load(open('/home/jp/projects/snoplus/python_modules/pmt_positions.pckl'))

In [4]:
quality_dir = '/home/jp/projects/snoplus/laserball_calibration/pmt_quality_occupancy'

## Speed of light

In [5]:
c = 0.299792458*1000 # mm/ns

## Getting SOC files - produce peaks files first

In [6]:
peakdir = '/home/jp/projects/snoplus/laserball_calibration/peak_output'

In [7]:
infile_list = os.listdir(peakdir)
for i, x in enumerate(infile_list): print i, x 

0 17386.pckl
1 17377.pckl
2 100558.pckl
3 100560.pckl
4 100559.pckl
5 17378.pckl
6 17375.pckl
7 100555.pckl
8 17384.pckl
9 17376.pckl
10 100556.pckl
11 100554.pckl


# Fit everything!

In [48]:
use_quality_cuts = False
use_peak = False # False means using dN/dt
if use_peak:
    data_index = 0
else:
    data_index = 1

In [49]:
fit_values = np.zeros([len(infile_list), 4])
for i, one_run in enumerate(infile_list):
    
    print 'Starting run', one_run
    
    # Get the data
    data = pickle.load(open(os.path.join(peakdir, one_run)))
    
    # Get the PMT quality
    pmtq_fname = os.path.join(quality_dir, one_run.rstrip('.pckl') + '_PMT_quality.pckl')
    pmtq_file = open(pmtq_fname)
    pmtq = pickle.load(pmtq_file)
    pmtq_file.close()
        
    # Select the PMTs that should be included in the fit
    typebool = (pmt_info['type'] == 1) + (pmt_info['type']==7)
    socpmts  = np.zeros_like(pmt_info['type'],dtype=bool)
    socpmts[data['soc_pmts']] = True
    nonzero_peaks = (data['delta_ts'][:,0] > 0)*(data['delta_ts'][:,1] > 0)
    mypmtbool = np.array(typebool*socpmts*nonzero_peaks,dtype=bool)

    print 'PMTs in the fit pre-Q-cuts', np.sum(mypmtbool)
    
    if use_quality_cuts:
        pmt_q_bool = (pmtq['positive']*pmtq['global_cut']*pmtq['normal_pmts']*
                      pmtq['nearby_cut']*pmtq['unobstructed'])
    else:
        pmt_q_bool = (pmtq['positive']*pmtq['unobstructed'])
        

    lbfit = FitLBpos( data = data['delta_ts'][:,data_index],
                      error = np.abs(data['delta_ts'][:,0]-data['delta_ts'][:,1]),
                      pmt_xyz = pmt_info['xyz'],
                      pmtbool = mypmtbool*pmt_q_bool,
                      print_call = False)
    
    wrapfcn = lambda p: lbfit(*p)
    repeat_fit = True
    
    fit_counter = 0
    while repeat_fit:
        my_x0 = np.concatenate((data['fit_pos']*(1+np.random.rand(3)/10.), [1.358]))
        fit = scipy.optimize.minimize(wrapfcn,
                                  x0 = my_x0,
                                  method='SLSQP',
                                  bounds=((-6000,6000),
                                          (-6000,6000),
                                          (-6000,6000),
                                          (0.5, 2.)),
                                  options={'ftol':1E-7, 'maxiter':1000})
        fit_counter += 1
        if fit.success:
            repeat_fit = False
            fit_values[i, : ] = fit.x
            print '\nFit successful', one_run.rstrip('.pckl')
            print data['manip_pos']
            print data['fit_pos']
            print fit.x, '\n'
        else:
            print 'Trying again', fit_counter
            if fit_counter >= 10:
                print 'Tried to fit TEN times without success - check that'
                fit_values[i,:] = [-1]*4
                repeat_fit = False
                
    
    
    

Starting run 17386.pckl
PMTs in the fit pre-Q-cuts 7649
Trying again 1
Trying again 2
Trying again 3
Trying again 4
Trying again 5
Trying again 6
Trying again 7
Trying again 8
Trying again 9
Trying again 10
Tried to fit TEN times without success - check that
Starting run 17377.pckl
PMTs in the fit pre-Q-cuts 2

Fit successful 17377
[   0. -254.   25.]
[   2.34015226 -226.16465759   24.06827164]
[  6.45557929e+02  -5.94279543e+02   4.64166968e+03   2.00000000e+00] 

Starting run 100558.pckl
PMTs in the fit pre-Q-cuts 8315

Fit successful 100558
[   0. -254.   25.]
[   6.7621541  -208.42858887   26.4679184 ]
[  -8.00106687 -233.08537461   14.13158467    1.35582073] 

Starting run 100560.pckl
PMTs in the fit pre-Q-cuts 17

Fit successful 100560
[   0. -254.   25.]
[   3.88579869 -207.37480164   27.52195358]
[ -2.25421418e+03   2.09636839e+02  -6.00000000e+03   6.09662080e-01] 

Starting run 100559.pckl
PMTs in the fit pre-Q-cuts 0

Fit successful 100559
[   0. -254.   25.]
[   2.24983931 

/home/jp/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.



Fit successful 17384
[   0. -254.   25.]
[  -1.41507101 -219.31124878   21.13463783]
[  23.1287987  -223.77817462  -16.85909869    1.37787584] 

Starting run 17376.pckl
PMTs in the fit pre-Q-cuts 0

Fit successful 17376
[   0. -254.   25.]
[   6.92900801 -225.00495911   24.29921532]
[   7.13740191 -238.99386814   24.59624244    1.358     ] 

Starting run 100556.pckl
PMTs in the fit pre-Q-cuts 8317

Fit successful 100556
[   0. -254.   25.]
[   3.82698131 -207.07707214   24.78620338]
[  12.63069986 -209.13405742    0.44959142    1.37481259] 

Starting run 100554.pckl
PMTs in the fit pre-Q-cuts 0

Fit successful 100554
[   0. -254.   25.]
[   1.49445307 -205.81411743   27.46838951]
[   1.59655644 -225.1215825    27.95520027    1.358     ] 



In [43]:
for i, fname in enumerate(infile_list):
    print fname.rstrip('.pckl'), '\t', '\t'.join(["%0.4f" % x for x in fit_values[i,:]])

17386 	180.2425	-11.4041	-1027.6274	0.7258
17377 	2.5145	-245.5338	24.0817	1.3000
100558 	-18.8864	-254.9390	34.8627	1.3430
100560 	-58.0867	-2350.9556	-1858.5849	0.8243
100559 	2.4010	-208.1592	27.6866	1.3000
17378 	8.5218	-242.6129	8.4931	1.3000
17375 	-11.4714	-239.0169	15.4847	1.3288
100555 	150.5715	657.4651	2014.0393	1.0409
17384 	24.3538	-283.7317	31.0462	1.3412
17376 	7.3428	-233.9207	26.1119	1.3000
100556 	7.1757	-229.0435	14.8138	1.3578
100554 	1.4979	-211.9499	27.6768	1.3000


In [ ]:
data